In [1]:
from sklearn.model_selection import KFold
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import pandas as pd
import torch.nn.functional as F

C:\Users\jonat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = pd.read_csv('train_data.csv')


In [4]:
indices = {}

cont = 0

for x in train_dataset['Etiqueta']:
    if x not in indices:
        indices[x] = cont
        cont+=1
        
train_dataset['Etiqueta'] = train_dataset['Etiqueta'].map(indices)
train_dataset

,Texto,Etiqueta
0,¿Puedo reservar un turno para la próxima semana?,0
1,Quisiera agendar una cita médica para el marte...,0
2,¿Están disponibles los turnos para el especial...,0
3,Necesito una cita con el odontólogo lo antes p...,0
4,¿Podría obtener un horario para una consulta d...,0
...,...,...
1052,seguro de salud de la institución,5
1053,seguro de seguro de salud aceptado por la inst...,5
1054,seguro de seguro médico que acepta la institución,5
1055,seguro de enfermedad aceptado por la clínica,5


In [11]:

# Configurar el modelo y el tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Configurar la validación cruzada K-Fold
kfold = KFold(n_splits=4, shuffle=True, random_state=42)

# Entrenamiento del modelo en cada fold
for train_index, val_index in kfold.split(train_dataset):
    train_data, val_data = train_dataset.iloc[train_index], train_dataset.iloc[val_index]
    
    
    # Procesamiento de los datos de entrenamiento
    train_encodings = tokenizer(train_dataset['Texto'].tolist(), truncation=True, padding=True, return_tensors='pt').to(device)
    train_labels = torch.tensor(train_dataset['Etiqueta'].tolist()).to(device)


    # Ajustar el modelo con los datos de entrenamiento
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)  # Ajusta el learning rate
    criterion = torch.nn.CrossEntropyLoss()  # Define la función de pérdida

    batch_size = 80  # Tamaño del batch
    num_epochs = 10  # Número de epochs

    train_loader = torch.utils.data.DataLoader(list(zip(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)), batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) 
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
         
        print(f"Epoch {epoch+1} Loss: {epoch_loss / len(train_loader)}")

model.save_pretrained('asistente') # ruta para guardar el modelo
tokenizer.save_pretrained('asistente') # ruta para guardar el tokenizer del modelo

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 Loss: 1.8798556625843048
Epoch 2 Loss: 1.8639002740383148
Epoch 3 Loss: 1.743060663342476
Epoch 4 Loss: 1.767170011997223
Epoch 5 Loss: 1.768181473016739
Epoch 6 Loss: 1.7620436251163483
Epoch 7 Loss: 1.7688672170042992
Epoch 8 Loss: 1.7568030655384064
Epoch 9 Loss: 1.7541908249258995
Epoch 10 Loss: 1.7502980157732964
Epoch 11 Loss: 1.7191012501716614
Epoch 12 Loss: 1.7467118948698044
Epoch 13 Loss: 1.7416902780532837
Epoch 14 Loss: 1.7323157414793968
Epoch 15 Loss: 1.7181469202041626
Epoch 1 Loss: 2.0557513535022736
Epoch 2 Loss: 1.7738358676433563
Epoch 3 Loss: 1.7643640413880348
Epoch 4 Loss: 1.7387757822871208
Epoch 5 Loss: 1.772527925670147
Epoch 6 Loss: 1.7556380331516266
Epoch 7 Loss: 1.7414259910583496
Epoch 8 Loss: 1.7524184808135033
Epoch 9 Loss: 1.7385631054639816
Epoch 10 Loss: 1.7352093905210495
Epoch 11 Loss: 1.750987447798252
Epoch 12 Loss: 1.7041757702827454
Epoch 13 Loss: 1.7812691032886505
Epoch 14 Loss: 1.729643851518631
Epoch 15 Loss: 1.7194742932915688


('asistente\\tokenizer_config.json',
 'asistente\\special_tokens_map.json',
 'asistente\\vocab.txt',
 'asistente\\added_tokens.json')

In [12]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import pandas as pd

# Suponiendo que tienes tus datos cargados en train_dataset

# Configurar el modelo y el tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Configurar la validación cruzada K-Fold
kfold = KFold(n_splits=6, shuffle=True, random_state=42)

best_validation_loss = float('inf')  # Inicializa la mejor pérdida de validación
best_model_path = 'best_model.pth'  # Ruta para guardar el mejor modelo

# Entrenamiento del modelo en cada fold
for train_index, val_index in kfold.split(train_dataset):
    train_data, val_data = train_dataset.iloc[train_index], train_dataset.iloc[val_index]

    # Procesamiento de los datos de entrenamiento
    train_encodings = tokenizer(train_data['Texto'].tolist(), truncation=True, padding=True, return_tensors='pt').to(device)
    train_labels = torch.tensor(train_data['Etiqueta'].tolist()).to(device)

    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=0.001,
        eps=1e-8,
        amsgrad=True,
        fused=True
    )
    criterion = torch.nn.CrossEntropyLoss()  # Define la función de pérdida

    batch_size = 70  # Tamaño del batch
    num_epochs = 15  # Número de epochs

    train_loader = torch.utils.data.DataLoader(list(zip(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)), batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # Proceso de validación
        model.eval()
        val_encodings = tokenizer(val_data['Texto'].tolist(), truncation=True, padding=True, return_tensors='pt').to(device)
        val_labels = torch.tensor(val_data['Etiqueta'].tolist()).to(device)

        with torch.no_grad():
            val_loader = torch.utils.data.DataLoader(list(zip(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)), batch_size=batch_size)
            val_loss = 0.0
            predictions = []

            for batch in val_loader:
                input_ids, attention_mask, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()

                logits = outputs.logits
                predictions.extend(logits.argmax(dim=-1).cpu().numpy())

        val_accuracy = accuracy_score(val_data['Etiqueta'], predictions)

        if val_loss < best_validation_loss:
            best_validation_loss = val_loss
            torch.save(model.state_dict(), best_model_path)

        print(f"Epoch {epoch+1} Training Loss: {epoch_loss / len(train_loader)} Validation Loss: {val_loss / len(val_loader)} Validation Accuracy: {val_accuracy}")

# Carga el mejor modelo guardado
best_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)
best_model.load_state_dict(torch.load(best_model_path))
best_model.to(device)

# Guardar el modelo y el tokenizer
best_model.save_pretrained('asistente')
tokenizer.save_pretrained('asistente')


KeyboardInterrupt: 

In [13]:
# Ruta donde has guardado el modelo
ruta_del_modelo = "asistente"

# Cargar el modelo y el tokenizador
model = BertForSequenceClassification.from_pretrained(ruta_del_modelo)
tokenizer = BertTokenizer.from_pretrained(ruta_del_modelo)

# Ejemplo de nueva entrada para hacer predicciones
nuevo_texto = "Quiero un turno"

# Tokenizar el texto de entrada
inputs = tokenizer(nuevo_texto, truncation=True, padding=True, return_tensors='pt')

# Obtener las predicciones del modelo
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits

# Aplicar la función softmax a los logits para obtener probabilidades
probabilities = F.softmax(predictions, dim=1)

etiqueta_idx = torch.argmax(probabilities)

print(etiqueta_idx)

tensor(0)


In [15]:
def intencion(texto):
    alm = {v: k for k, v in indices.items()}
    inputs = tokenizer(texto, truncation=True, padding=True, return_tensors='pt')

    # Obtener las predicciones del modelo
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits

    # Aplicar la función softmax a los logits para obtener probabilidades
    probabilities = F.softmax(predictions, dim=1)

    etiqueta_idx = torch.argmax(probabilities)
    
    return alm[etiqueta_idx.item()]

In [16]:
consultas = {
    'consulta_turno': [
        '¿Puedo obtener un turno para una consulta con el Dr. Smith?',
        'Quisiera programar una cita médica para el próximo lunes, ¿es posible?',
        '¿Están disponibles los turnos para pediatría mañana por la tarde?'
    ],
    'pregunta_horario': [
        '¿Cuál es el horario de atención del departamento de emergencias?',
        '¿A qué hora abre la clínica los sábados?',
        '¿Hasta qué hora están recibiendo pacientes hoy?'
    ],
    'pregunta_especialidad': [
        '¿Podrían decirme si tienen un especialista en cardiología disponible?',
        '¿Quién es el neurólogo de guardia hoy?',
        '¿Tienen médicos especializados en traumatología?'
    ],
    'consulta_resultados': [
        'Me gustaría saber si mis resultados de análisis de sangre ya están disponibles.',
        '¿Dónde puedo ver los resultados de mi radiografía?',
        '¿Los resultados de mi ecografía ya están listos?'
    ],
    'pregunta_receta': [
        '¿Puedo solicitar una receta para mi medicamento?',
        'Necesito una receta para renovar mi medicación, ¿cómo puedo obtenerla?',
        '¿El médico puede enviarme la receta por correo electrónico?'
    ],
    'consulta_seguro': [
        '¿Aceptan mi seguro médico para consultas en esta clínica?',
        '¿Qué tipos de seguros médicos son aceptados aquí?',
        '¿Pueden informarme sobre los procedimientos cubiertos por mi seguro de salud?'
    ],
    'pregunta_certificado': [
        '¿Es posible obtener un certificado médico para faltar al trabajo?',
        '¿Cómo puedo obtener un certificado de aptitud física?',
        '¿Pueden proporcionarme un certificado para justificar una ausencia escolar?'
    ],
    'pregunta_emergencia': [
        'Tengo una emergencia médica, ¿qué debo hacer?',
        '¿Dónde está ubicado el área de emergencias?',
        '¿Cuál es el número de contacto para emergencias fuera del horario de atención?'
    ]
}


In [17]:
for x,y in consultas.items():
    for e in y:
        print(x,' : ','Prediccion : ',intencion(e), ' | ',e)

consulta_turno  :  Prediccion :  consulta_turno  |  ¿Puedo obtener un turno para una consulta con el Dr. Smith?
consulta_turno  :  Prediccion :  consulta_turno  |  Quisiera programar una cita médica para el próximo lunes, ¿es posible?
consulta_turno  :  Prediccion :  consulta_turno  |  ¿Están disponibles los turnos para pediatría mañana por la tarde?
pregunta_horario  :  Prediccion :  consulta_turno  |  ¿Cuál es el horario de atención del departamento de emergencias?
pregunta_horario  :  Prediccion :  consulta_turno  |  ¿A qué hora abre la clínica los sábados?
pregunta_horario  :  Prediccion :  consulta_turno  |  ¿Hasta qué hora están recibiendo pacientes hoy?
pregunta_especialidad  :  Prediccion :  consulta_turno  |  ¿Podrían decirme si tienen un especialista en cardiología disponible?
pregunta_especialidad  :  Prediccion :  consulta_turno  |  ¿Quién es el neurólogo de guardia hoy?
pregunta_especialidad  :  Prediccion :  consulta_turno  |  ¿Tienen médicos especializados en traumatolog

In [ ]:
intencion('Tienen medicos clinicos en el hospital?')

'consulta_turno'